In [1]:
import torch
%load_ext autoreload
%autoreload 2
model_dict = torch.load('/usr/src/app/radtts/outputs/30shuff_sdfixed_dap_test_checkpoint_5000.pt',map_location=torch.device('cpu'))



In [2]:
import sys
import importlib
spec = importlib.util.spec_from_file_location("exp_config", '/usr/src/app/uberduck_ml_exp/configs/radtts/30_spkr_shuffle_zero_dap.py')
foo = importlib.util.module_from_spec(spec)
sys.modules["exp_config"] = foo
spec.loader.exec_module(foo)
from exp_config import config
# from uberduck_ml_dev.exec.train_radtts_with_ray import config
from uberduck_ml_dev.models.radtts import RADTTS


In [3]:
# from uberduck_ml_dev.exec.train_radtts_with_ray import model_config
model_config = config['model_config']
model = RADTTS(
    **model_config,
)

Applying spectral norm to text encoder LSTM
Applying spectral norm to context encoder LSTM


/usr/src/app/uberduck_ml_dev/uberduck_ml_dev/models/common.py:1527: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2349.)
  W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
/opt/conda/lib/python3.8/site-packages/torch/functional.py:1682: UserWarning: torch.lu is deprecated in favor of torch.linalg.lu_factor / torch.linalg.lu_factor_ex and will be removed in a future PyTorch release.
LU, pivots = torch.lu(A, compute_pivots)
should be replaced with
LU, pivots = torch.linalg.lu_factor(A, compute_pivots)
and
LU, pivots, info = torch.lu(A, compute_pivots, get_infos=True)
should be replaced with
LU, pivots, info = torch.linalg.lu_factor_ex(A, compute_pivots) (Triggered

In [4]:
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in model_dict['state_dict'].items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
# load params
model.load_state_dict(new_state_dict)
model.eval()
model.cuda()

RADTTS(
  (speaker_embedding): Embedding(30, 512)
  (embedding): Embedding(185, 512)
  (flows): ModuleList(
    (0): FlowStep(
      (invtbl_conv): Invertible1x1ConvLUS()
      (affine_tfn): AffineTransformationLayer(
        (affine_param_predictor): WN_RADTTS(
          (in_layers): ModuleList(
            (0): ConvNorm(
              (conv): PartialConv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
            )
            (1): ConvNorm(
              (conv): PartialConv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(4,), dilation=(2,))
            )
            (2): ConvNorm(
              (conv): PartialConv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(8,), dilation=(4,))
            )
            (3): ConvNorm(
              (conv): PartialConv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(16,), dilation=(8,))
            )
          )
          (res_skip_layers): ModuleList(
            (0): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))


In [5]:
%tb
RESNET_SE_MODEL_PATH = '/usr/src/app/radtts/resnet_se.pth.tar'
RESNET_SE_CONFIG_PATH = '/usr/src/app/radtts/resnet_se_config.json'

import json
with open(RESNET_SE_CONFIG_PATH) as f:
    resnet_config = json.load(f)
state_dict = torch.load(RESNET_SE_MODEL_PATH)['model']
audio_config = dict(resnet_config["audio"])
model_params = resnet_config["model_params"]
if "model_name" in model_params:
    del model_params["model_name"]
from TTS.encoder.models.resnet import ResNetSpeakerEncoder
se = ResNetSpeakerEncoder(**model_params, audio_config=audio_config)
se.load_state_dict(state_dict)
se.eval()
se.cpu()

import pandas as pd
from uberduck_ml_dev.exec.train_radtts_with_ray import load_wav_to_torch, get_text, get_energy_average, get_attention_prior
data = pd.read_csv(
    '/usr/src/app/radtts/data/30_decoder_pitch.txt',
    sep="|",
    header=None,
    quoting=3,
    names = ['path', 'transcript', 'speaker_id', 'f0_path', 'emb_path']
)

No traceback available to show.


{'train_config': {'output_directory': '', 'epochs': 10000, 'optim_algo': 'RAdam', 'learning_rate': 0.0005, 'weight_decay': 1e-06, 'sigma': 1.0, 'iters_per_checkpoint': 5000, 'steps_per_sample': 2000, 'batch_size': 32, 'seed': None, 'checkpoint_path': '', 'ignore_layers': [], 'ignore_layers_warmstart': [], 'finetune_layers': [], 'include_layers': [], 'vocoder_config_path': '/usr/src/app/radtts/models/hifigan_22khz_config.json', 'vocoder_checkpoint_path': '/usr/src/app/radtts/models/hifigan_libritts100360_generator0p5.pt', 'log_attribute_samples': True, 'log_decoder_samples': True, 'warmstart_checkpoint_path': '/usr/src/app/radtts/outputs/30shuff_sdfixed_test_checkpoint_60000.pt', 'use_amp': False, 'grad_clip_val': 1.0, 'loss_weights': {'blank_logprob': -1, 'ctc_loss_weight': 0.1, 'binarization_loss_weight': 1.0, 'dur_loss_weight': 1.0, 'f0_loss_weight': 1.0, 'energy_loss_weight': 1.0, 'vpred_loss_weight': 1.0}, 'binarization_start_iter': 0, 'kl_loss_start_iter': 0, 'unfreeze_modules': '

In [6]:
import numpy as np
from uberduck_ml_dev.data.audio_processing import TacotronSTFT
from uberduck_ml_dev.models.common import get_mel
import os
stft = TacotronSTFT(
    filter_length=data_config['filter_length'],
    hop_length=data_config['hop_length'],
    win_length=data_config['win_length'],
    sampling_rate=22050,
    n_mel_channels=data_config['n_mel_channels'],
    mel_fmin=data_config['mel_fmin'],
    mel_fmax=data_config['mel_fmax'],
)


,path,transcript,speaker_id,f0_path,emb_path
0,/usr/src/app/radtts/data/TAG/wavs/1_22k_normal...,"Let me kick it, fire spitting, all these words...",0,/usr/src/app/radtts/data_cache/data_TAG_wavs_1...,NaN
1,/usr/src/app/radtts/data/TAG/wavs/2_22k_normal...,"Living day to day, trying to make a right and ...",0,/usr/src/app/radtts/data_cache/data_TAG_wavs_2...,NaN
2,/usr/src/app/radtts/data/TAG/wavs/3_22k_normal...,"But I'm sinning with my brother, with eachothe...",0,/usr/src/app/radtts/data_cache/data_TAG_wavs_3...,NaN
3,/usr/src/app/radtts/data/TAG/wavs/4_22k_normal...,"Let me talk, prove the walk, no my stride won'...",0,/usr/src/app/radtts/data_cache/data_TAG_wavs_4...,NaN
4,/usr/src/app/radtts/data/TAG/wavs/5_22k_normal...,"Till I'm out here being great, I'm the chosen ...",0,/usr/src/app/radtts/data_cache/data_TAG_wavs_5...,NaN
...,...,...,...,...,...
21797,/usr/src/app/radtts/data/Po_Data/wavs/231_22k_...,I defeated Tai Lung.,22,/usr/src/app/radtts/data_cache/data_Po_Data_wa...,NaN
21798,/usr/src/app/radtts/data/Po_Data/wavs/232_22k_...,"No, Master!",22,/usr/src/app/radtts/data_cache/data_Po_Data_wa...,NaN
21799,/usr/src/app/radtts/data/Po_Data/wavs/233_22k_...,No no no! Don't die Shifu please!,22,/usr/src/app/radtts/data_cache/data_Po_Data_wa...,NaN
21800,/usr/src/app/radtts/data/Po_Data/wavs/234_22k_...,"Oh, so um, should stop talking?",22,/usr/src/app/radtts/data_cache/data_Po_Data_wa...,NaN


In [19]:
import numpy as np
from uberduck_ml_dev.exec.train_radtts_with_ray import get_vocoder
vocoder = get_vocoder()

Getting vocoder
Loading pretrained model...
Got pretrained model...


In [61]:

sigma = .8
sigma_tkndur = 0.666
sigma_f0 = 1.0
sigma_energy = 1.0
f0_mean = 0.0
f0_std = 1.
energy_mean = 0.0
energy_std = 0.0
token_dur_scaling = 1.2
n_takes = 1
use_amp = True
plot = True
seed = 1234
kwargs = dict(sigma = .8,
sigma_tkndur = 0.666,
sigma_f0 = 1.0,
sigma_energy = 1.0,
f0_mean = 0.0,
f0_std = 1.,
energy_mean = 0.0,
energy_std = 0.0,
n_takes = 1,
use_amp = True,
plot = True,
seed = 1234)

# config_path = "/usr/src/app/radtts/models/hifigan_22khz_config.json"
# vocoder_path = "/usr/src/app/radtts/models/hifigan_libritts100360_generator0p5.pt"
# vocoder_path = "/usr/src/app/radtts/herring-2-9-23-hifigan"
# vocoder, denoiser = load_vocoder(vocoder_path, config_path, to_cuda=True)
# denoising_strength = 0.

In [62]:
def generate_guided_inference(guide_radtts, decoder_radtts, guide_audio_embedding, decoder_audio_embedding, text_encoded):
    
    guide_output = get_guide_outputs(guide_radtts, guide_audio_embedding,text_encoded)
    output = get_final_outputs(guide_output, decoder_radtts, decoder_audio_embedding, text_encoded)    
    return output

def get_guide_outputs(guide_radtts, guide_audio_embedding,text,  **kwargs):
    
    # not used
    speaker_id = torch.tensor([0])
    speaker_id_text, speaker_id_attributes = speaker_id, speaker_id
    outputs = guide_radtts.infer(speaker_id, text, sigma, sigma_tkndur, sigma_f0,
                            sigma_energy, token_dur_scaling, token_duration_max=100,
                            speaker_id_text=speaker_id_text,
                            speaker_id_attributes=speaker_id_attributes,
                            f0_mean=f0_mean, f0_std=f0_std, energy_mean=energy_mean,
                            energy_std=energy_std, audio_embedding = guide_audio_embedding)
    return outputs



def get_final_outputs(guide_outputs, decoder_model, decoder_audio_embedding,text):
    
    # not used
    speaker_id = torch.tensor([0])
    
    durs = guide_outputs['dur']
    energy_avg= guide_outputs['energy_avg']
    target_length = None
    new_durs = _get_durs(durs, target_length)

    speaker_id_text, speaker_id_attributes = speaker_id, speaker_id

    outputs = decoder_model.infer(speaker_id, text, sigma, sigma_tkndur, sigma_f0,
                            sigma_energy, dur = new_durs,
                                  token_dur_scaling = 1.2,
                            speaker_id_text=speaker_id_text,
                            speaker_id_attributes=None, audio_embedding = decoder_audio_embedding)    
    return outputs

def _get_durs(durs, target_length = None):

    if target_length is None:
        return durs
    else:
        current_durs = durs.sum()
        new_durs = durs.float() * target_length / current_durs
        new_durs = new_durs.round().long()
        return new_durs
    

In [63]:
i = 0
audio, sample_rate = load_wav_to_torch(data['path'].iloc[i])
audio_norm = audio / (np.abs(audio).max() * 2)
print(data['path'].iloc[i])
text = data['transcript'].iloc[i]
print(text)

/usr/src/app/radtts/data/TAG/wavs/1_22k_normalized.wav
Let me kick it, fire spitting, all these words be hitting different.


In [64]:
emb_list = []
# text_list = []
for i in [500,1000,10000]:
    path = data['path'].iloc[i]
    print(path)
    audio, sample_rate = load_wav_to_torch(path)
    audio_norm = audio / (np.abs(audio).max() * 2)
    datum = torch.FloatTensor(audio_norm).unsqueeze(-1).t()
    emb = se(datum)
    audio_embedding = emb.cpu().detach().numpy()
    audio_embedding = torch.FloatTensor(audio_embedding)
    emb_list.append(audio_embedding)

/usr/src/app/radtts/data/eminem/wav22050/274_22k_normalized.wav
/usr/src/app/radtts/data/sam-lachow/wavs/79_22k_normalized.wav
/usr/src/app/radtts/data/lj_data/LJSpeech-1.1/wavs/LJ028-0325_22k_normalized.wav


In [65]:
from uberduck_ml_dev.exec.train_radtts_with_ray import get_text
from einops import rearrange

In [66]:
speaker_ids = None # should be unused
text = "The problem with substring searches is that, in this example, you're looking for the word is inside This be a string. That will evaluate to True because of the"
text_sequence = get_text(text)

text_sequence = rearrange(text_sequence, 'i -> 1 i')
text_sequence = torch.LongTensor(text_sequence)

In [67]:
model.cuda()
def mel_to_audio(mel_out):
    
    
    audio = vocoder(mel_out).float()[0]
#     audio_denoised = denoiser(
#         audio, strength=denoising_strength)[0].float()

    audio = audio[0].cpu().detach().numpy()
#     audio_denoised = audio_denoised[0].cpu().detach().numpy()
#     audio_denoised = audio_denoised / np.max(np.abs(audio_denoised))
    return audio


In [68]:
import IPython.display as ipd

In [73]:
output = generate_guided_inference(guide_radtts = model,
                          decoder_radtts = model,
                          guide_audio_embedding = emb_list[0].cuda(),
                          decoder_audio_embedding = emb_list[2].cuda(),
                          text_encoded = text_sequence.cuda())

ipd.Audio(mel_to_audio(output['mel'].cpu()), rate = 22050)